# Analysis of company furniture data

We aim to get 
1. Top products
2. Top clients
3. Overall revenue performance

In [1]:
import pandas as pd

# Load the datasets
customers = pd.read_csv('datasets/data1/customers.csv')
products = pd.read_csv('datasets/data1/products.csv')
ratings = pd.read_csv('datasets/data1/ratings.csv')
orders = pd.read_csv('datasets/data1/orders.csv')

In [2]:
# Merge the datasets
merged_data = orders.merge(customers, left_on='customer_id', right_on='customer_id') \
                    .merge(products, left_on='product_id', right_on='product_id') \
                    .merge(ratings, on=['customer_id', 'product_id'], how='left')


In [3]:
# Calculate revenue for each product
merged_data['revenue'] = merged_data['price'] * merged_data['quantity']


In [4]:
# Top performing products in terms of revenue
top_products_revenue = merged_data.groupby('product_name')['revenue'].sum().reset_index() \
                                  .sort_values('revenue', ascending=False)
print("Top performing products in terms of revenue:")
print(top_products_revenue.head())

Top performing products in terms of revenue:
           product_name  revenue
5   Brimnes Bed Storage    59521
41     Småstad Wardrobe    51688
37         Råskog Stool    50445
6         Docksta Table    46386
27         Nockeby Sofa    45646


In [5]:
# Top performing products in terms of units sold
top_products_units = merged_data.groupby('product_name')['quantity'].sum().reset_index() \
                                .sort_values('quantity', ascending=False)
print("\nTop performing products in terms of units sold:")
print(top_products_units.head())


Top performing products in terms of units sold:
               product_name  quantity
22    Mackapar Shoe Storage        80
5       Brimnes Bed Storage        77
31           Poäng Armchair        63
45   Söderhamn Sofa Section        63
1   Bekant Conference Table        62


In [8]:
merged_data['order_date'] = pd.to_datetime(merged_data['order_date'])

In [11]:
# Top clients for the last month
last_month = pd.to_datetime('today') - pd.DateOffset(months=10) # previous 10 months
top_clients_last_month = merged_data[merged_data['order_date'] > last_month] \
                                     .groupby('name')['revenue'].sum().reset_index() \
                                     .sort_values('revenue', ascending=False)
print("\nTop clients for the last month:")
print(top_clients_last_month.head())


Top clients for the last month:
           name  revenue
0    Customer_1    27081
39  Customer_44    26529
34   Customer_4    25717
22  Customer_29    24997
48  Customer_52    24055


In [12]:
customer_orders = merged_data.groupby('customer_id').agg({'order_id': 'count', 'revenue': 'sum'}).reset_index()
customer_orders.columns = ['customer_id', 'order_count', 'total_revenue']
customer_orders

,customer_id,order_count,total_revenue
0,1,18,27081
1,2,9,9347
2,3,13,18077
3,4,22,25717
4,5,11,14913
...,...,...,...
95,96,9,12264
96,97,4,6088
97,98,7,8915
98,99,10,13181


In [14]:
# Customer segmentation based on order count, total revenue, and recency
customer_orders = merged_data.groupby('customer_id').agg({
    'order_id': 'count',
    'revenue': 'sum',
    'order_date': 'max'
}).reset_index()

customer_orders.columns = ['customer_id', 'order_count', 'total_revenue', 'most_recent_order']

# Calculate recency (days since last order)
import datetime
now = datetime.datetime.now()
customer_orders['recency'] = (now - customer_orders['most_recent_order']).dt.days

# Segment customers based on order count, total revenue, and recency
order_count_quartiles = customer_orders['order_count'].quantile([0.25, 0.5, 0.75])
revenue_quartiles = customer_orders['total_revenue'].quantile([0.25, 0.5, 0.75])
recency_quartiles = customer_orders['recency'].quantile([0.25, 0.5, 0.75])

customer_orders['order_count_segment'] = pd.qcut(customer_orders['order_count'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
customer_orders['revenue_segment'] = pd.qcut(customer_orders['total_revenue'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
customer_orders['recency_segment'] = pd.qcut(customer_orders['recency'], 4, labels=['Very Recent', 'Recent', 'Not Recent', 'Very Not Recent'])

print('Customer Segmentation:')
print(customer_orders)

Customer Segmentation:
    customer_id  order_count  total_revenue          most_recent_order  \
0             1           18          27081 2023-09-17 08:39:23.971834   
1             2            9           9347 2023-09-15 08:39:23.971834   
2             3           13          18077 2023-09-13 08:39:23.971834   
3             4           22          25717 2023-09-14 08:39:23.971834   
4             5           11          14913 2023-09-09 08:39:23.971834   
..          ...          ...            ...                        ...   
95           96            9          12264 2023-09-15 08:39:23.971834   
96           97            4           6088 2023-09-09 08:39:23.971834   
97           98            7           8915 2023-09-11 08:39:23.971834   
98           99           10          13181 2023-09-17 08:39:23.971834   
99          100           12          20744 2023-09-06 08:39:23.971834   

    recency order_count_segment revenue_segment  recency_segment  
0       195          